# Test of clustering metrics
Quick sanity checks for the metrics on a near-uniform random configuration.

In [ ]:
import os, sys
sys.path.append(os.path.abspath('..'))

import numpy as np
from src.metrics import nearest_neighbor_distance, largest_cluster_fraction

box_size = 1.0
eps = 0.05

for i in range(5):
    positions = np.random.rand(100, 2) * box_size
    nn = nearest_neighbor_distance(positions, box_size=box_size)
    lcf = largest_cluster_fraction(positions, eps=eps, box_size=box_size)
    print(f'Run {i}: NN={nn:.3f}, LCF={lcf:.2f}')
